In [1]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import allel
import seaborn as sns
import allel
import pickle
from scipy.stats import spearmanr, pearsonr, kendalltau

from scipy.stats import pearsonr


In [2]:
e_wc = pd.read_csv('../gwas/worldclim_ecotypesdata_sorted_20240517.csv', sep = '\t')[['ecotypeid','bio1']]

In [3]:
train_mean = np.mean(e_wc['bio1'])
train_std = np.std(e_wc['bio1'])

##scale it
e_wc['bio1'] = (e_wc['bio1'] - np.mean(e_wc['bio1'])) / np.std(e_wc['bio1'])

In [4]:
#sites_climate = pd.read_csv('../../grene/data/worldclim_sitesdata.csv')[['site', 'bio1']]
sites_climate = pd.read_csv('../key_files/bioclimvars_experimental_sites_era5.csv')[['site', 'bio1']]

sites_climate['bio1'] = (sites_climate['bio1'] - train_mean) / train_std

In [5]:
# Extract climate data as numpy arrays
climate_ecotypes = e_wc['bio1'].to_numpy()
climate_sites = sites_climate['bio1'].to_numpy()

# Calculate the absolute differences between each pair of ecotype and site
climatic_distance_matrix = (climate_ecotypes[:, np.newaxis] - climate_sites)**2

# Convert the distance matrix back to a DataFrame for better readability
distance_df = pd.DataFrame(climatic_distance_matrix, index=e_wc['ecotypeid'], columns=sites_climate['site'])


In [6]:
ecotypes_names = distance_df.index

In [7]:
#path_ecotypes = '/carnegie/nobackup/scratch/xwu/grenet/merged_frequency/merged_ecotype_frequency.txt'
#ecotypes_names = pd.read_csv('/carnegie/nobackup/scratch/xwu/grenet/ecotypes_names.txt',header=None)[0].to_list()
final_gen = pd.read_csv('../key_files/final_gen.csv')['sample_name']
#real_ef = pd.read_csv(path_ecotypes, sep = '\t',usecols = final_gen)
#real_ef.index = ecotypes_names
final_gen = final_gen[~final_gen.str.startswith('33_')]


In [8]:
unique_sites = final_gen.str.split('_').str[0].unique()

In [9]:
final_gen = final_gen.to_list()

In [10]:
#real_ef=pd.read_csv('../leave_1_out/stabilizing_selection_data_scaled_2024Jun18.txt', sep = '\t')[['log_p1_p0','ecotype', 'site','plot']]


In [11]:
delta_ef=pd.read_csv('../key_files/delta_ecotype_freq.txt', sep = '\t', usecols = final_gen)#[['log_p1_p0','ecotype', 'site','plot']]
ef=pd.read_csv('../key_files/merged_ecotype_frequency.txt', sep = '\t', usecols = final_gen)#[['log_p1_p0','ecotype', 'site','plot']]
delta_ef.index = ecotypes_names
ef.index = ecotypes_names

In [12]:
#samples = pd.read_csv('../key_files/merged_sample_table.csv')[['site', 'plot', 'generation', 'total_flower_counts']]

#samples = samples.groupby(['site', 'plot', 'generation'])['total_flower_counts'].sum().reset_index()

#samples['min_perc'] = 1/samples['total_flower_counts']

#samples['code'] = samples['site'].astype(str) + '_'  + samples['generation'].astype(str) + '_' + samples['plot'].astype(str) 

#ecotype_freq = pd.read_csv('../key_files/merged_ecotype_frequency.txt',sep = '\t')

#climate = pd.read_csv('../key_files/bioclimvars_experimental_sites_era5.csv')[['site', 'bio1']]

#for i in ecotype_freq.columns:
    #min_perc = samples[samples['code'] == i]['min_perc'].values[0]
#    total_flower_counts = samples[samples['code'] == i]['total_flower_counts'].values[0]
    #ecotype_freq.loc[ecotype_freq[i] < min_perc, i] = np.nan
#    ecotype_freq[i] = ecotype_freq[i] * total_flower_counts

#ecotype_freq = ecotype_freq[final_gen]

#delta_ef = ecotype_freq.copy()

In [13]:
#delta_ef.index = ecotypes_names

In [215]:
## P1/P0 ECOTYPE FREQ
#real_ef['sample'] = real_ef['site'].astype(str) + '_' + real_ef['plot'].astype(str) 
#real_ef['p1_over_p0'] = np.exp(real_ef['log_p1_p0'])
#real_ef = real_ef.drop(['log_p1_p0', 'site', 'plot'],axis=1)
#real_ef = real_ef.pivot_table(index = 'ecotype', columns = 'sample', values='p1_over_p0')
#real_ef.index = real_ef.index.astype(int)

In [216]:
#path_ecotypes = '../stabilizing_selection_data.txt'

#real_ef = pd.read_csv(path_ecotypes, sep = '\t') #,usecols = final_gen)

#unique_sites = real_ef['site'].unique()

#real_ef['sample'] = real_ef['site'].astype(str) + '_' + real_ef['plot'].astype(str) 

#real_ef = real_ef[['log_p1_p0', 'ecotype', 'sample']]



#real_ef = real_ef.drop(real_ef[real_ef['log_p1_p0'] == -np.inf].index)

#real_ef = real_ef.pivot_table(index = 'ecotype', columns = 'sample', values='log_p1_p0')

In [217]:

#real_ef = real_ef.sort_index()

In [14]:
import pickle

# Path to your .pkl file
file_path = '../jacknife_lfmm/splits_samples_last_gen_no33.pkl'

# Open and load the .pkl file
with open(file_path, 'rb') as file:
    splits_samples_first_gen = pickle.load(file)

In [15]:
len(splits_samples_first_gen)

30

In [16]:
all_splits_predictions = {}
for split in range(len(splits_samples_first_gen)):
    print(split)
    ## ge thte train and test samples/sites
    train = splits_samples_first_gen[split][0]
    
    test = splits_samples_first_gen[split][1]
    ##get the sites to train/test 
    sites_u_train = pd.Series(train).str.split('_').str[0].unique()
    sites_u_test = pd.Series(test).str.split('_').str[0].unique()
    # conver tthem to int for easiest manipulations
    sites_u_train_int =  [int(i) for i in sites_u_train ]
    sites_u_test_int =  [int(i) for i in sites_u_test ]
    ## get the necesary cliamtic distances and ecotype freq 
    ef1 = ef[[col for col in ef.columns if any(col.startswith(f'{site}_') for site in sites_u_train_int)]]
    cd = distance_df[sites_u_train_int]

    cd_test = distance_df[sites_u_test_int]
    cd_test = cd_test.reset_index().melt(id_vars=['ecotypeid'])
    
    # melt them for easy manipulation
    ef_train = ef1.reset_index().melt(id_vars = ['ecotypeid'])
    cd_train = cd.reset_index().melt(id_vars = ['ecotypeid'])
    
    ef_train['site'] = ef_train['variable'].str.split('_').str[0]
    ef_train = ef_train.drop('variable',axis=1)
    
    ef_train.columns = ['ecotypeid', 'ef', 'site']
    cd_train.columns = ['ecotypeid', 'site', 'cd']
    
    cd_train['site'] = cd_train['site'].astype(str)

    ## get the full trianign dataset with all ecotypes 
    train = ef_train.merge(cd_train, on =['site', 'ecotypeid'])

    ## predict each ecotype at a time 
    prediction_ecotypes = {}
    for ecotype_1 in train['ecotypeid'].unique():
        
        model1 = train[train['ecotypeid'] == ecotype_1]
        
        X = model1['cd'].values.reshape(-1, 1)  # Independent variable
        y = model1['ef'].values # Dependent variable
        
        # Fit the linear model
        model = LinearRegression().fit(X, y)
        # Predict the valu.es
        
        cd_ecotype = cd_test[cd_test['ecotypeid'] == ecotype_1]['value'].values[0]
        y_pred = model.predict(cd_ecotype.reshape(1, -1) )
    
        prediction_ecotypes[ecotype_1] = y_pred
    
    prediction_ecotypes = pd.DataFrame(prediction_ecotypes).T
    
    all_splits_predictions[sites_u_test[0]] = prediction_ecotypes

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [17]:
all_splits_predictions = pd.concat(all_splits_predictions,axis=1)

In [18]:
all_splits_predictions.columns = all_splits_predictions.columns.get_level_values(0)

In [19]:
results_mean = {}
for split in range(len(splits_samples_first_gen)):
    ## ge thte train and test samples/sites    
    test = splits_samples_first_gen[split][1]
    ##get the sites to train/test 
    sites_u_test = pd.Series(test).str.split('_').str[0].unique()
    # conver tthem to int for easiest manipulations
    sites_u_test_int =  [int(i) for i in sites_u_test ]
    ## get the necesary cliamtic distances and ecotype freq 
    ef1 = ef[[col for col in ef.columns if any(col.startswith(f'{site}_') for site in sites_u_test_int)]]
    ef_mean = ef1.mean(axis=1)
    ef_mean.name = 'mean'
    ## select the prediciotn to compare 
    prediction_to_test = all_splits_predictions[sites_u_test[0]]

    conc = pd.concat([ef_mean, prediction_to_test],axis=1)
    ## becasue some ecotpyes are nan 
    
    X_ranked = conc[sites_u_test[0]]
    y_ranked = conc['mean']

    sp_correlation, _ = spearmanr(X_ranked, y_ranked)
    pearsonr_value = pearsonr(X_ranked, y_ranked)[0]
    kendall_tau, _ = kendalltau(X_ranked, y_ranked)

    X = conc[sites_u_test[0]].values.reshape(-1, 1)  # Independent variable
    y = conc['mean'].values # Dependent variable

    # Fit the linear model
    model = LinearRegression().fit(X, y)
    # Predict the valu.es
    y_pred = model.predict(X)
    
    r_squared = r2_score(y, y_pred)
            # Find the top 10 indices for observed and predicted value
    res_list = [sp_correlation, pearsonr_value, r_squared, kendall_tau]
    for i in [10,20,30,40,50]:
        ## TAIL BEcause the most common ecotpyes are at thebottom, sor values does ascending 
        top_10_observed_indices =  conc['mean'].sort_values().tail(i).index
        top_10_predicted_indices = conc[sites_u_test[0]].sort_values().tail(i).index

        # Compare top 10 indices
        common_top_indices = len(set(top_10_observed_indices).intersection(set(top_10_predicted_indices)))

        # Calculate the percentage of common top 10 indices
        top_10_match_percentage = (common_top_indices / i) * 100
        res_list.append(top_10_match_percentage)
    results_mean[sites_u_test[0]] = res_list

In [22]:
results_mean = pd.DataFrame(results_mean).T

results_mean.columns = ['sp_correlation', 'pearsonr', 'r_squared','kendall_tau', 'top_10','top_20','top_30','top_40','top_50']

results_mean['sp_correlation'] = results_mean['sp_correlation'] 

results_mean['pearsonr'] = results_mean['pearsonr'] 


results_mean = results_mean.reset_index()

results_mean = results_mean.rename(columns = {'index':'site'})

In [24]:
results_mean.to_csv('climate_distance_model_results_mean_leave_1_out.csv',index=None)

In [25]:
results = {}
for split in range(len(splits_samples_first_gen)):
    ## ge thte train and test samples/sites    
    test = splits_samples_first_gen[split][1]
    ##get the sites to train/test 
    sites_u_test = pd.Series(test).str.split('_').str[0].unique()
    # conver tthem to int for easiest manipulations
    sites_u_test_int =  [int(i) for i in sites_u_test ]
    ## get the necesary cliamtic distances and ecotype freq 
    ef1 = ef[[col for col in ef.columns if any(col.startswith(f'{site}_') for site in sites_u_test_int)]]

    ## select the prediciotn to compare 
    prediction_to_test = all_splits_predictions[sites_u_test[0]]
    for sample in ef1.columns:
        
        ef_1sample = ef1[sample]
        conc = pd.concat([ef_1sample, prediction_to_test],axis=1)
        ## becasue some ecotpyes are nan 
        
        X_ranked = conc[sites_u_test[0]]
        y_ranked = conc[sample]

        sp_correlation, _ = spearmanr(X_ranked, y_ranked)
        pearsonr_value = pearsonr(X_ranked, y_ranked)[0]
        kendall_tau, _ = kendalltau(X_ranked, y_ranked)

        X = conc[sites_u_test[0]].values.reshape(-1, 1)  # Independent variable
        y = conc[sample].values # Dependent variable

        # Fit the linear model
        model = LinearRegression().fit(X, y)
        # Predict the valu.es
        y_pred = model.predict(X)
        
        r_squared = r2_score(y, y_pred)
                # Find the top 10 indices for observed and predicted values

        res_list = [sp_correlation, pearsonr_value, r_squared, kendall_tau]

        for i in [10,20,30,40,50]:
            top_10_observed_indices = conc[sample].sort_values().tail(i).index
            top_10_predicted_indices = conc[sites_u_test[0]].sort_values().tail(i).index
    
            # Compare top 10 indices
            common_top_indices = len(set(top_10_observed_indices).intersection(set(top_10_predicted_indices)))
    
            # Calculate the percentage of common top 10 indices
            top_10_match_percentage = (common_top_indices / i) * 100
            res_list.append(top_10_match_percentage)
        results[sample] = res_list

In [27]:
results = pd.DataFrame(results).T

results.columns = ['sp_correlation', 'pearsonr', 'r_squared','kendall_tau', 'top_10','top_20','top_30','top_40','top_50']

#results.columns = ['sp_correlation', 'pearsonr', 'r_squared', 'top_10_match_percentage', 'kendall_tau']

results = results.reset_index()

results['site'] = results['index'].str.split('_').str[0]

results['plot'] = results['index'].str.split('_').str[2]

In [28]:
results

,index,sp_correlation,pearsonr,r_squared,kendall_tau,top_10,top_20,top_30,top_40,top_50,site,plot
0,1_2_2,0.460552,0.358252,0.128344,0.315114,30.0,30.0,26.666667,32.5,36.0,1,2
1,1_2_3,0.443606,0.118825,0.014119,0.302014,10.0,25.0,23.333333,30.0,36.0,1,3
2,1_3_1,0.270915,0.115559,0.013354,0.185067,10.0,20.0,20.000000,32.5,42.0,1,1
3,1_3_4,0.195153,0.060537,0.003665,0.128020,20.0,10.0,10.000000,15.0,26.0,1,4
4,1_3_5,0.395064,0.277638,0.077083,0.264044,10.0,20.0,20.000000,22.5,32.0,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...
343,60_1_8,0.437185,0.120607,0.014546,0.291766,10.0,15.0,26.666667,30.0,30.0,60,8
344,60_1_9,0.120858,0.069203,0.004789,0.082447,10.0,10.0,16.666667,17.5,28.0,60,9
345,60_1_10,0.321997,0.197681,0.039078,0.222536,30.0,25.0,20.000000,25.0,30.0,60,10
346,60_1_11,0.386155,0.227219,0.051628,0.268620,10.0,30.0,26.666667,32.5,42.0,60,11


In [29]:
results.to_csv('climate_distance_model_results_leave_1_out_ecotype_freq_top.csv',index=None)

In [ ]:
#### NO MODEL JUST NAIVE CORRECLATION, NO INFORMATION ABOUT GRENENT 

In [31]:
results_mean = {}
for site in unique_sites:
    ## one site 
    ## retreive ecotype frequecnies 
    ef1 = ef[[col for col in ef.columns if col.startswith(f'{site}_')]]
    cd = distance_df[int(site)]
    ef_f = ef1.mean(axis=1)
    ef_f.name = 'mean'
    conc = pd.concat([ef_f, cd],axis=1)
    conc = conc.sort_values('mean')
    ## becasue some ecotpyes are nan 
    conc = conc.dropna()
    X_ranked = conc['mean']
    y_ranked = conc[int(site)]

    sp_correlation, _ = spearmanr(X_ranked, y_ranked)
    pearsonr_value = pearsonr(X_ranked, y_ranked)[0]
    kendall_tau, _ = kendalltau(X_ranked, y_ranked)

    X = conc[int(site)].values.reshape(-1, 1)  # Independent variable
    y = conc['mean'].values # Dependent variable

    # Fit the linear model
    model = LinearRegression().fit(X, y)
    # Predict the valu.es
    y_pred = model.predict(X)
    
    r_squared = r2_score(y, y_pred)
            # Find the top 10 indices for observed and predicted values

    res_list = [sp_correlation, pearsonr_value, r_squared, kendall_tau]

    for i in [10,20,30,40,50]:
        # head becasue climatics distance is bigger when thee cotypesshould do worst 
        top_10_observed_indices = conc['mean'].sort_values().tail(i).index
        top_10_predicted_indices = conc[int(site)].sort_values().head(i).index

        # Compare top 10 indices
        common_top_indices = len(set(top_10_observed_indices).intersection(set(top_10_predicted_indices)))

        # Calculate the percentage of common top 10 indices
        top_10_match_percentage = (common_top_indices / i) * 100
        res_list.append(top_10_match_percentage)
    results_mean[site] = res_list
    ## retreive climatic distances  


In [32]:
results_mean = pd.DataFrame(results_mean).T

In [33]:

results_mean.columns = ['sp_correlation', 'pearsonr', 'r_squared','kendall_tau', 'top_10','top_20','top_30','top_40','top_50']

results_mean['sp_correlation'] = results_mean['sp_correlation'] *-1

results_mean['pearsonr'] = results_mean['pearsonr']  *-1
results_mean['kendall_tau'] = results_mean['kendall_tau']  *-1
results_mean = results_mean.reset_index()

#results.columns = ['sp_correlation', 'pearsonr', 'r_squared', 'top_10_match_percentage', 'kendall_tau']

results_mean = results_mean.rename(columns = {'index': 'site'})

In [34]:
results_mean.to_csv('naive_climate_distance_top_ecotypes_mean.csv',index=None)

In [35]:
results = {}
for site in unique_sites:
    ## one site 
    ## retreive ecotype frequecnies 
    ef1 = ef[[col for col in ef.columns if col.startswith(f'{site}_')]]
    cd = distance_df[int(site)]
    
    for sample in ef1.columns:
        ef_f = ef1[sample]
        conc = pd.concat([ef_f, cd],axis=1)
        conc = conc.sort_values(sample)
        ## becasue some ecotpyes are nan 
        conc = conc.dropna()
        X_ranked = conc[sample]
        y_ranked = conc[int(site)]

        sp_correlation, _ = spearmanr(X_ranked, y_ranked)
        pearsonr_value = pearsonr(X_ranked, y_ranked)[0]
        kendall_tau, _ = kendalltau(X_ranked, y_ranked)

        X = conc[int(site)].values.reshape(-1, 1)  # Independent variable
        y = conc[sample].values # Dependent variable

        # Fit the linear model
        model = LinearRegression().fit(X, y)
        # Predict the valu.es
        y_pred = model.predict(X)
        
        r_squared = r2_score(y, y_pred)
                # Find the top 10 indices for observed and predicted values

        res_list = [sp_correlation, pearsonr_value, r_squared, kendall_tau]

        for i in [10,20,30,40,50]:
            top_10_observed_indices = conc[sample].sort_values().tail(i).index
            top_10_predicted_indices = conc[int(site)].sort_values().head(i).index
    
            # Compare top 10 indices
            common_top_indices = len(set(top_10_observed_indices).intersection(set(top_10_predicted_indices)))
    
            # Calculate the percentage of common top 10 indices
            top_10_match_percentage = (common_top_indices / i) * 100
            res_list.append(top_10_match_percentage)
        results[sample] = res_list
    ## retreive climatic distances  


In [36]:
results = pd.DataFrame(results).T

results.columns = ['sp_correlation', 'pearsonr', 'r_squared','kendall_tau', 'top_10','top_20','top_30','top_40','top_50']

results['sp_correlation'] = results['sp_correlation'] *-1

results['pearsonr'] = results['pearsonr']  *-1
results['kendall_tau'] = results['kendall_tau']  *-1
results = results.reset_index()

#results.columns = ['sp_correlation', 'pearsonr', 'r_squared', 'top_10_match_percentage', 'kendall_tau']

results = results.reset_index()

results['site'] = results['index'].str.split('_').str[0]

results['plot'] = results['index'].str.split('_').str[2]

In [37]:
results

,level_0,index,sp_correlation,pearsonr,r_squared,kendall_tau,top_10,top_20,top_30,top_40,top_50,site,plot
0,0,1_2_2,0.408645,0.110037,0.012108,0.293025,10.0,25.0,20.000000,20.0,22.0,1,2
1,1,1_2_3,0.449537,0.096117,0.009239,0.316596,0.0,5.0,20.000000,20.0,34.0,1,3
2,2,1_3_1,0.503553,0.041096,0.001689,0.364814,0.0,20.0,26.666667,37.5,40.0,1,1
3,3,1_3_4,0.382439,0.056012,0.003137,0.263813,0.0,10.0,23.333333,35.0,42.0,1,4
4,4,1_3_5,0.349472,0.110924,0.012304,0.245777,0.0,15.0,20.000000,27.5,24.0,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,343,60_1_8,0.251909,0.015499,0.000240,0.174604,0.0,20.0,30.000000,30.0,30.0,60,8
344,344,60_1_9,0.045511,0.010238,0.000105,0.015553,0.0,15.0,23.333333,20.0,28.0,60,9
345,345,60_1_10,0.233987,0.071336,0.005089,0.167731,0.0,10.0,26.666667,25.0,26.0,60,10
346,346,60_1_11,0.643584,0.162564,0.026427,0.500212,10.0,40.0,63.333333,65.0,66.0,60,11


In [38]:
results.to_csv('naive_climate_distance_top_ecotypes.csv',index=None)